# WEB SCRAPING AMAZON'S BEST SELLER DATA USING PYTHON

In [16]:
# importing necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
import csv

# Scraping data using ScraperAPI and BeautifulSoup 

In [17]:
def scrape_page(category, link):
    url = link
    payload = {'api_key': 'eb4e3ae733167f3076b822fa3ca8aa03', 'url': url}
    page = requests.get('http://api.scraperapi.com', params = payload)

    soup = BeautifulSoup(page.content, 'html')
        
    i=0

    product_names = []
    product_category = []
    product_prices = []
    product_reviews = []
    product_ratings = []
    product_rank = []
    product_link = []
    date = []
    
    while i<30:   
        card = soup.find_all(id=f"p13n-asin-index-{i}")

        for c in card:
            name = c.find_all(class_=lambda class_name: class_name and class_name.startswith('_cDEzb_p13n-sc-css-line-clamp-'))
            price = c.find_all(class_='_cDEzb_p13n-sc-price_3mJ9Z')
            reviews = c.find_all(class_='a-size-small')
            ratings = c.find_all(class_='a-icon-alt')
            rank = c.find_all(class_="zg-bdg-text")
            link = c.find_all('a',class_='a-link-normal')
    #       ------------------------------------------------------------------------------------
            if name:
                if category in ['Apps for Android', 'Movies & TV Shows', 'Music']:
                    for n in name[::2]:
                        product_names.append(n.get_text())
                elif category in['Software','Video Games']:
                    for n in name[::3]:
                        product_names.append(n.get_text())
                else:
                    for n in name:
                        product_names.append(n.get_text())
            else:
                product_names.append('null')
    #       ------------------------------------------------------------------------------------
            product_category.append(category)
            date.append(datetime.date.today())
    #       ------------------------------------------------------------------------------------
            if price:
                for p in price:
                    product_prices.append(p.get_text())
            else:
                product_prices.append('null')
    #       ------------------------------------------------------------------------------------
            if reviews:
                contains_digit = any(item.get_text().replace(",","").isdigit() for item in reviews)
                if contains_digit:
                    for r in reviews:
                        if len(r['class'])==1:
                            product_reviews.append(r.get_text())
                else:
                    product_reviews.append("Null")

    #       ------------------------------------------------------------------------------------
            if ratings:
                for r in ratings:
                    product_ratings.append(r.get_text())
            else:
                product_ratings.append('null')
    #       ------------------------------------------------------------------------------------
            if rank:
                for r in rank:
                    product_rank.append(r.get_text())
            else:
                product_rank.append('null')
    #       ------------------------------------------------------------------------------------
            if link:
                for l in link:
                    product_link.append(f'amazon.in{l["href"]}')
            else:
                product_link.append('null')

            filtered_links1 = []

            for link in product_link:
                if link not in filtered_links1:
                    filtered_links1.append(link)

            filtered_links2 = [link for link in filtered_links1 if 'review' not in link]
            product_link.clear()
            product_link = filtered_links2

        i+=1
    book_names = []
    if category == "Books" or category == 'Kindle Store':
        for i in product_names[::2]:
            author = product_names[product_names.index(i)+1]
            book_names.append(f'{i}, Author: {author}')
        product_names.clear()
        product_names = book_names
        
    
    rows = zip(product_names,product_category,product_prices,product_reviews,product_ratings,product_rank,product_link,date)

    with open('AmazonBestSellers.csv', 'a', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            for row in rows:
                writer.writerow(row)
    

In [18]:
header = ['Product Name','Category','Price','Number of Reviews','Average Rating','Product Rank','Product URL','Date']

with open('AmazonBestSellers.csv','w', newline='', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [19]:
category = {
    'Amazon Launchpad': 'https://www.amazon.in/gp/bestsellers/boost/ref=zg_bs_nav_boost_0',
    'Amazon Renewed': 'https://www.amazon.in/gp/bestsellers/amazon-renewed/ref=zg_bs_nav_amazon-renewed_0',
    'Apps for Android': 'https://www.amazon.in/gp/bestsellers/mobile-apps/ref=zg_bs_nav_mobile-apps_0',
    'Baby Products': 'https://www.amazon.in/gp/bestsellers/baby/ref=zg_bs_nav_baby_0' ,
    'Bags, Wallets and Luggage': 'https://www.amazon.in/gp/bestsellers/luggage/ref=zg_bs_luggage_sm',
    'Beauty': 'https://www.amazon.in/gp/bestsellers/beauty/ref=zg_bs_nav_beauty_0',
    'Books': 'https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_nav_books_0',
    'Car & Motorbike': 'https://www.amazon.in/gp/bestsellers/automotive/ref=zg_bs_nav_automotive_0',
    'Clothing & Accessories': 'https://www.amazon.in/gp/bestsellers/apparel/ref=zg_bs_apparel_sm',
    'Computers & Accessorties': 'https://www.amazon.in/gp/bestsellers/computers/ref=zg_bs_nav_computers_0',
    'Electronics':"https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_electronics_sm",
    'Garden & Outdoors': 'https://www.amazon.in/gp/bestsellers/garden/ref=zg_bs_nav_garden_0',
    'Gift Cards': 'https://www.amazon.in/gp/bestsellers/gift-cards/ref=zg_bs_nav_gift-cards_0',
    'Grocery & Gourmet Foods': 'https://www.amazon.in/gp/bestsellers/grocery/ref=zg_bs_nav_grocery_0',
    'Health & Personal Care':'https://www.amazon.in/gp/bestsellers/hpc/ref=zg_bs_nav_hpc_0' ,
    'Home & Kitchen': 'https://www.amazon.in/gp/bestsellers/kitchen/ref=zg_bs_nav_kitchen_0',
    'Home Improvement':'https://www.amazon.in/gp/bestsellers/home-improvement/ref=zg_bs_nav_home-improvement_0' ,
    'Industrial & Scientific': 'https://www.amazon.in/gp/bestsellers/industrial/ref=zg_bs_nav_industrial_0',
    'Jewellery':'https://www.amazon.in/gp/bestsellers/jewelry/ref=zg_bs_nav_jewelry_0' ,
    'Kindle Store':'https://www.amazon.in/gp/bestsellers/digital-text/ref=zg_bs_nav_digital-text_0' ,
    'Movies & TV Shows': 'https://www.amazon.in/gp/bestsellers/dvd/ref=zg_bs_nav_dvd_0',
    'Music': 'https://www.amazon.in/gp/bestsellers/music/ref=zg_bs_nav_music_0' ,
    'Musical Instruments': 'https://www.amazon.in/gp/bestsellers/musical-instruments/ref=zg_bs_nav_musical-instruments_0',
    'Office Products': 'https://www.amazon.in/gp/bestsellers/office/ref=zg_bs_nav_office_0' ,
    'Pet Supplies': 'https://www.amazon.in/gp/bestsellers/pet-supplies/ref=zg_bs_nav_pet-supplies_0',
    'Shoes & Handbags': 'https://www.amazon.in/gp/bestsellers/shoes/ref=zg_bs_nav_shoes_0' ,
    'Software':'https://www.amazon.in/gp/bestsellers/software/ref=zg_bs_nav_software_0' ,
    'Sports, Fitness & Outdoors': 'https://www.amazon.in/gp/bestsellers/sports/ref=zg_bs_nav_sports_0',
    'Toys & Games': 'https://www.amazon.in/gp/bestsellers/toys/ref=zg_bs_nav_toys_0' ,
    'Video Games': 'https://www.amazon.in/gp/bestsellers/videogames/ref=zg_bs_nav_videogames_0',
    'Watches': 'https://www.amazon.in/gp/bestsellers/watches/ref=zg_bs_watches_sm' 
}

for category_name, category_url in category.items():
    scrape_page(category_name, category_url)

# Data Transformation in Pandas

In [32]:
pd.set_option('display.max.rows', 925)
df = pd.read_csv(r"C:\Users\Mahima Parekh\AmazonBestSellers.csv")
df

,Product Name,Category,Price,Number of Reviews,Average Rating,Product Rank,Product URL,Date
0,SOFTSPUN Microfiber Cloth - 4 pcs - 40x40 cms ...,Amazon Launchpad,₹269.00,"76,418",4.3 out of 5 stars,#1,amazon.in/SOFTSPUN-Microfiber-Cleaning-Detaili...,2023-09-07
1,Desidiya ® 20 Photo Clip LED String Lights for...,Amazon Launchpad,₹199.00,"21,211",4.1 out of 5 stars,#2,amazon.in/DesiDiya-Birthday-Festival-Restauran...,2023-09-07
2,Fixderma Shadow Sunscreen SPF 30+ Gel Sunscree...,Amazon Launchpad,₹186.00,"4,755",4.3 out of 5 stars,#3,amazon.in/Fixderma-Shadow-SPF-Gel-40gm/dp/B089...,2023-09-07
3,"Be Bodywise Biotin Hair Gummies for Stronger, ...",Amazon Launchpad,₹908.00,"6,955",4.0 out of 5 stars,#4,amazon.in/Be-Bodywise-Supplements-Strawberry-M...,2023-09-07
4,"SOFTSPUN Microfiber Cleaning Cloths, 4pcs 40x4...",Amazon Launchpad,₹247.00,"22,498",4.3 out of 5 stars,#5,amazon.in/SOFTSPUN-Microfiber-Cleaning-Cloth-3...,2023-09-07
5,Sirona Reusable Menstrual Cup for Women | Medi...,Amazon Launchpad,₹258.00,"25,593",4.3 out of 5 stars,#6,amazon.in/Sirona-Approved-Reusable-Menstrual-S...,2023-09-07
6,INDIGENOUS HONEY Raw Organic Honey NMR Tested ...,Amazon Launchpad,₹521.00,"14,211",4.3 out of 5 stars,#7,amazon.in/Indigenous-Unprocessed-Unfiltered-Un...,2023-09-07
7,ALPINO Natural Peanut Butter Crunch 400g - 100...,Amazon Launchpad,₹149.00,"26,043",4.2 out of 5 stars,#8,amazon.in/Alpino-Natural-Peanut-Unsweetened-No...,2023-09-07
8,"Frido Dual Gel Heavy Duty Trimmable Insoles, F...",Amazon Launchpad,₹799.00,"3,410",3.8 out of 5 stars,#9,amazon.in/Frido-Trimmable-Insoles-Replacing-Ex...,2023-09-07
9,Man Matters Biotin Hair Gummies | 60 Days Pack...,Amazon Launchpad,₹799.00,"4,169",3.8 out of 5 stars,#10,amazon.in/Man-Matters-Strawberry-Flavoured-Veg...,2023-09-07


In [21]:
#deleting unnecessary columns
df1 = df.drop(columns = ['Date','Product URL'])

In [46]:
df1

,Product Name,Category,Price,Number of Reviews,Average Rating,Product Rank
0,SOFTSPUN Microfiber Cloth - 4 pcs - 40x40 cms ...,Amazon Launchpad,269.00,76418.0,4.3,1
1,Desidiya ® 20 Photo Clip LED String Lights for...,Amazon Launchpad,199.00,21211.0,4.1,2
2,Fixderma Shadow Sunscreen SPF 30+ Gel Sunscree...,Amazon Launchpad,186.00,4755.0,4.3,3
3,"Be Bodywise Biotin Hair Gummies for Stronger, ...",Amazon Launchpad,908.00,6955.0,4.0,4
4,"SOFTSPUN Microfiber Cleaning Cloths, 4pcs 40x4...",Amazon Launchpad,247.00,22498.0,4.3,5
5,Sirona Reusable Menstrual Cup for Women | Medi...,Amazon Launchpad,258.00,25593.0,4.3,6
6,INDIGENOUS HONEY Raw Organic Honey NMR Tested ...,Amazon Launchpad,521.00,14211.0,4.3,7
7,ALPINO Natural Peanut Butter Crunch 400g - 100...,Amazon Launchpad,149.00,26043.0,4.2,8
8,"Frido Dual Gel Heavy Duty Trimmable Insoles, F...",Amazon Launchpad,799.00,3410.0,3.8,9
9,Man Matters Biotin Hair Gummies | 60 Days Pack...,Amazon Launchpad,799.00,4169.0,3.8,10


In [23]:
#removing the rs symbol from column Price
df1['Price']=df1['Price'].str.strip("₹")

In [24]:
#removing the , from column Number of Reviews
df1['Number of Reviews'] = df1['Number of Reviews'].str.replace(',','')

In [25]:
#removing the 'out of 5 stars' from column Average Rating
df1['Average Rating'] = df1['Average Rating'].str.strip(' out of 5 stars')

In [26]:
#removing the #symbol from column Product Rank
df1['Product Rank']=df1['Product Rank'].str.strip('#')

In [39]:
df1['Price'] = pd.to_numeric(df1['Price'], errors='coerce')

In [40]:
df1['Number of Reviews'] = pd.to_numeric(df1['Number of Reviews'], errors='coerce')

In [41]:
df1['Average Rating'] = pd.to_numeric(df1['Average Rating'], errors='coerce')

In [42]:
df1['Product Rank'] = pd.to_numeric(df1['Product Rank'], errors='coerce')

In [48]:
df1.dtypes

Product Name          object
Category              object
Price                float64
Number of Reviews    float64
Average Rating       float64
Product Rank           int64
dtype: object

In [47]:
df1

,Product Name,Category,Price,Number of Reviews,Average Rating,Product Rank
0,SOFTSPUN Microfiber Cloth - 4 pcs - 40x40 cms ...,Amazon Launchpad,269.00,76418.0,4.3,1
1,Desidiya ® 20 Photo Clip LED String Lights for...,Amazon Launchpad,199.00,21211.0,4.1,2
2,Fixderma Shadow Sunscreen SPF 30+ Gel Sunscree...,Amazon Launchpad,186.00,4755.0,4.3,3
3,"Be Bodywise Biotin Hair Gummies for Stronger, ...",Amazon Launchpad,908.00,6955.0,4.0,4
4,"SOFTSPUN Microfiber Cleaning Cloths, 4pcs 40x4...",Amazon Launchpad,247.00,22498.0,4.3,5
5,Sirona Reusable Menstrual Cup for Women | Medi...,Amazon Launchpad,258.00,25593.0,4.3,6
6,INDIGENOUS HONEY Raw Organic Honey NMR Tested ...,Amazon Launchpad,521.00,14211.0,4.3,7
7,ALPINO Natural Peanut Butter Crunch 400g - 100...,Amazon Launchpad,149.00,26043.0,4.2,8
8,"Frido Dual Gel Heavy Duty Trimmable Insoles, F...",Amazon Launchpad,799.00,3410.0,3.8,9
9,Man Matters Biotin Hair Gummies | 60 Days Pack...,Amazon Launchpad,799.00,4169.0,3.8,10


In [50]:
df1.to_csv('Amazon_Web_Scraping_Cleaned_Data.csv', index=False)  # Set index=False to exclude the row index in the CSV file